# Import bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

np.set_printoptions(precision=12, suppress=True, linewidth=120)
print(tf.__version__)

# Przygotowanie i eksploracja danych

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
print(X_train[0])

In [ ]:
print(f'X_train[0] shape: {X_train[0].shape}')

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
plt.imshow(X_train[0], cmap='gray_r')
plt.axis('off')

In [ ]:
plt.figure(figsize=(13, 13))
for i in range(1, 11):
    plt.subplot(1, 10, i)
    plt.axis('off')
    plt.imshow(X_train[i-1], cmap='gray_r')
    plt.title(y_train[i-1], color='white', fontsize=16)
plt.show()

# Budowa sieci neuronowej

In [ ]:
def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()

# Trenowanie modelu

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

# ModelCheckpoint - Zapisywanie wag po każdej poprawie modelu
Za każdym razem gdy monitorowana metryka ulega poprawie wagi są zapisywane do pliku.
Wykorzystamy klasę **ModelCheckpoint** do obserwowania metryki **accuracy** na zbiorze walidacyjnym.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

!mkdir model
filepath ='model/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

# ModelCheckpoint - Zapisanie najlepszego modelu

In [ ]:
filepath = 'best_model_weights.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

In [ ]:
best_model = Sequential()
best_model.add(Flatten(input_shape=(28, 28)))
best_model.add(Dense(units=128, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(units=10, activation='softmax'))

best_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

best_model.load_weights('best_model_weights.hdf5')

test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)

# Early Stopping - wczesne zatrzymanie
Klasa **EarlyStopping** pozwala na wcześniejsze zatrzymanie procesu trenowania sieci, gdy obserwowna metryka zacznie się pogarszać.

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, batch_size=32)

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.head()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['val_loss'], name='val_loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.add_trace(go.Scatter(y=metrics['val_accuracy'], name='val_accuracy'), row=2, col=1)
fig.update_layout(width=800)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# domyślnie mode='auto'
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

model = build_model()
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, batch_size=32, callbacks=[es])

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.head()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['val_loss'], name='val_loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.add_trace(go.Scatter(y=metrics['val_accuracy'], name='val_accuracy'), row=2, col=1)
fig.update_layout(width=800)

# Tensorboard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime as dt

log_dir = 'logs/fit/' + dt.now().strftime('%Y%m%d-%H%M%S')
tensorboard = TensorBoard(log_dir=log_dir)

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=3, validation_split=0.2, batch_size=32, callbacks=[tensorboard])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit